In [1]:
import numpy as np
from numpy import newaxis
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import sys
import librosa
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from tensorflow.ragged import constant

In [2]:
PATH = '/home/ferreiraa/Documents/Mestrado/trabalho/redeNeural/agender_distribution/'

In [3]:
def Feat_Conc(V1,V2):
    Vetor_Final = []
    if len(V1) == len(V2):
        for i in range(0,len(V1)):
            Vetor_Final.append([])
            for j in range(0,len(V1[i])):
                Vetor_Final[i].append(V1[i][j])
            for j in range(0,len(V2[i])):
                Vetor_Final[i].append(V2[i][j])

    return Vetor_Final

In [4]:
def Norm_size(V,n_carac=183,m_size=500):
    V2 = np.full((m_size,n_carac),fill_value=-500.0)
    #print(V.shape)
    if len(V) > m_size:
        Linhas = m_size
    else:
        Linhas = len(V)
    Colunas = n_carac
    for line in range(0,Linhas-1):
        for col in range(0,Colunas-1):
            if line < len(V) and col < len(V[0]):
                V2[line][col] = V[line][col]
    return V2

In [5]:
def Adapt_2_RNN(V,size=1000):
    Sup_Vec = np.array([V])
    Adapted_Vec = np.repeat(Sup_Vec,size,axis=0)
    return Adapted_Vec

In [6]:
Batch = 128
Epoch = 100

#Read files and split result from file
Train_file = pd.read_csv("Csv/train.csv")
Test_file = pd.read_csv("Csv/test.csv")
Arq_train = Train_file['file']
Arq_test = Test_file['file']
Res_train = Train_file['class']
Res_test = Test_file['class']

V_train_x = []
for i in Arq_train:
    if i !='file':
        V_train_x.append(i)

V_test_x = []
for i in Arq_test:
    if i !='file':
        V_test_x.append(i)

train_df = pd.DataFrame(V_train_x)
test_df = pd.DataFrame(V_test_x)

train_df = train_df.rename(columns={0:'file'})
test_df = test_df.rename(columns={0:'file'})

In [7]:
features_train = []
for i in range(0, len(train_df)):
    a = np.load(PATH + train_df['file'][i] + '.npz')
    features_train.append(np.concatenate((
        a['mfccs'],
        a['Delta_Mfcc'],
        a['contrast'],
        a['mel'],
        a['tonnetz'],
        a['ZCR'],
        a['RMS']), axis=0))
    
features_test = []
for i in range(0, len(test_df)):
    b = np.load(PATH + test_df['file'][i] + '.npz')
    features_test.append(np.concatenate((
        b['mfccs'],
        b['Delta_Mfcc'],
        b['contrast'],
        b['mel'],
        b['tonnetz'],
        b['ZCR'],
        b['RMS']), axis=0))

In [8]:
X_test = np.array(features_test)
X_train = np.asarray(features_train)
Y_train = np.array(Res_train)

lb = LabelEncoder()
Y_train = to_categorical(lb.fit_transform(Y_train))

X_train = X_train.reshape(len(X_train),223,1)
X_test = X_test.reshape(len(X_test),223,1)

In [9]:
input = 223
input_shape = 1

In [10]:
InpSp = input
Mod_Shape = input_shape

In [11]:
HU = int(InpSp * 3) # Hiddent units (First Hidden Layer)
model = Sequential()

# Usar Masking para -500.0
model.add(Masking(mask_value=-500.0,input_shape=(Mod_Shape,InpSp)))
#model.add(Masking(mask_value=-500.0,input_shape=(InpSp,)))
# Recurrent layer
model.add(LSTM(HU,return_sequences=False, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(InpSp*2, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(InpSp, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.15))

# Output layer
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
ini_train = time.time()
history = model.fit(X_train,  Y_train,batch_size=int(Batch), epochs=int(Epoch),validation_split=0.2)#, callbacks=[early_stop])
fim_train = time.time() - ini_train

Epoch 1/100


ValueError: in user code:

    /home/ferreiraa/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/ferreiraa/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ferreiraa/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ferreiraa/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ferreiraa/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ferreiraa/.local/lib/python3.8/site-packages/tensorflow/py

In [ ]:
N_epochs = int(epoch)
file = open("RNNTestes.txt",'a+')
file.write("Teste com "+str(Epoch)+" epocas e "+str(Batch)+" de batch: \n")
file.write("Tempo de Retirada de caracteristicas: "+str((1.0*fim_feat)/len(Y_test))+'\n')
file.write("Tempo de Treino: "+str(fim_train)+'\n')


train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']# Set figure size.
plt.figure(figsize=(12, 8))# Generate line plot of training, testing loss over epochs.
plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')# Set title
plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Categorical Crossentropy', fontsize = 18)
plt.xticks(range(0,int(Epoch),int(Epoch/20)), range(0,int(Epoch),int(Epoch/20)))
plt.legend(fontsize = 18);

plt.show()

In [ ]:
ini_test = time.time()
predictions = np.argmax(model.predict(X_test), axis=-1)
#predictions = lb.inverse_transform(predictions)
test_predict = predictions
fim_test = time.time() - ini_test

In [ ]:
file.write("Tempo de Teste: "+str((1.0*fim_test)/len(Res_test))+'\n')

correct_predict = 0.0
for i in range(0,len(Res_test)):
    if str(test_predict[i]) != 'NaN' and int(test_predict[i]) == int(Y_test[i]):
        correct_predict += 1.
print("A porcentagem de acerto é de : "+str((correct_predict*100)/len(Y_test))+"%")
file.write("Porcentagem de acerto: "+str((correct_predict*100)/len(Y_test))+"%\n")
file.write('##########\n')
file.close()